In [4]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CCSA'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [5]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2022_levantamento_CCSA.xlsx')
time.sleep(random.randint(60, 80))

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-08-31_14-41_CCSA_relatorio_almox.xlsx')
time.sleep(random.randint(60, 80))

logger.info(f'tratamdp dados da planilha de levantameto fisico {n_campi}')
time.sleep(random.randint(60, 80))

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

print(levantamento.info())
print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
time.sleep(random.randint(60, 80))
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')
time.sleep(random.randint(60, 80))

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

logger.info(f'consolidando dados')
time.sleep(random.randint(60, 80))

logger.info(f'exportando planilha de consolidação')
time.sleep(random.randint(60, 80))

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  268 non-null    datetime64[ns]
 1   Endereço de e-mail                    268 non-null    object        
 2   Retificação                           2 non-null      object        
 3   Código                                268 non-null    object        
 4   Nome                                  268 non-null    object        
 5   Quantidade                            268 non-null    int64         
 6   Campus                                268 non-null    object        
 7   Localização                           174 non-null    object        
 8   Valor unitário                        268 non-null    float64       
 9   Sem identificação - SI                0 non-null      float64       
 10  Ma

,codigo,descricao_suap,und,qtd_suap,ed
0,30.07.1462,Açucar cristal 1KG,UN,93,07 - GENEROS DE ALIMENTACAO
1,30.07.1464,Café em pó 250g,UN,208,07 - GENEROS DE ALIMENTACAO
2,30.07.451,"ADOÇANTE, ASPECTO FÍSICO LÍQUIDO SUCRALOSE.",UN,10,07 - GENEROS DE ALIMENTACAO
3,30.16.1065,"ESTOJO PARA DVD, EM PVC 19 X 13,5 X 1,3CM - TR...",UN,195,16 - MATERIAL DE EXPEDIENTE
4,30.16.1069,PRANCHETA 1/2 OFÍCIO CRISTAL C/ PRENDEDOR PLÁS...,UN,22,16 - MATERIAL DE EXPEDIENTE
...,...,...,...,...,...
259,30.42.12,BROCA DE VIDIA 12MM,UN,5,42 - FERRAMENTAS
260,30.42.213,BROCA DE AÇO RÁPIDO 10MM.,UN,5,42 - FERRAMENTAS
261,30.42.789,BROCA DE AÇO RÁPIDO 12 MM,UN,5,42 - FERRAMENTAS
262,30.42.790,BROCA AÇO RÁPIDO DE 8 MM,UN,5,42 - FERRAMENTAS


,codigo,descricao_lev,qtd_lev,servidor
0,30.16.1954,"BARBANTE ALGODÃO, QUANTIDADE FIOS 6 UNIDADES, ...",1,adriana.carmo@cabo.ifpe.edu.br
1,30.16.537,"FITA DUREX ADESIVA, MATERIAL CELOFANE TRANPARE...",19,atlog@cabo.ifpe.edu.br
2,30.16.1841,"CANETA DESENHO ARQUITETONICO, MATERIAL CORPO P...",7,adriana.carmo@cabo.ifpe.edu.br
3,30.36.481,"FITA , MATERIAL PVC, TIPO AUTO-ZEBRADA, 5cm x ...",28,atlog@cabo.ifpe.edu.br
4,30.16.1437,"MINA GRAFITE, DIÂMETRO 0,50 MM, COMPRIMENTO 60...",40,dap@cabo.ifpe.edu.br
...,...,...,...,...
263,30.24.3488,"PORTA, PADRÃO MADEIRA CEDRO, TIPO LISA, ACABAM...",2,atlog@cabo.ifpe.edu.br
264,30.26.1410,"30.26.1409 CABO ELETRICO FLEXIVEL, MATERIAL CO...",1,atlog@cabo.ifpe.edu.br
265,30.26.3180,"CABO ELÉTRICO - 4,2 MM, COR PRETO, FLEXÍVEL",1,atlog@cabo.ifpe.edu.br
266,30.26.3182,"CABO ELÉTRICO - 4,2 MM, COR AMARELA, FLEXÍVEL",1,atlog@cabo.ifpe.edu.br


In [6]:
logger.info(f'montando planilha de ajustes ')
time.sleep(random.randint(60, 80))

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')
time.sleep(random.randint(60, 80))

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.07.1462,Açucar cristal 1KG,UN,93,07 - GENEROS DE ALIMENTACAO,Açucar cristal 1KG,93,job.costa@cabo.ifpe.edu.br,both,0
1,30.07.1464,Café em pó 250g,UN,208,07 - GENEROS DE ALIMENTACAO,Café em pó 250g,208,job.costa@cabo.ifpe.edu.br,both,0
2,30.07.451,"ADOÇANTE, ASPECTO FÍSICO LÍQUIDO SUCRALOSE.",UN,10,07 - GENEROS DE ALIMENTACAO,"ADOÇANTE, ASPECTO FÍSICO LÍQUIDO SUCRALOSE.",10,job.costa@cabo.ifpe.edu.br,both,0
3,30.16.1065,"ESTOJO PARA DVD, EM PVC 19 X 13,5 X 1,3CM - TR...",UN,195,16 - MATERIAL DE EXPEDIENTE,"ESTOJO PARA DVD, EM PVC 19 X 13,5 X 1,3CM - TR...",195,dap@cabo.ifpe.edu.br,both,0
4,30.16.1069,PRANCHETA 1/2 OFÍCIO CRISTAL C/ PRENDEDOR PLÁS...,UN,22,16 - MATERIAL DE EXPEDIENTE,PRANCHETA 1/2 OFÍCIO CRISTAL C/ PRENDEDOR PLÁS...,22,atlog@cabo.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...,...
260,30.42.12,BROCA DE VIDIA 12MM,UN,5,42 - FERRAMENTAS,BROCA DE VIDIA 12MM,5,adriana.carmo@cabo.ifpe.edu.br,both,0
261,30.42.213,BROCA DE AÇO RÁPIDO 10MM.,UN,5,42 - FERRAMENTAS,BROCA DE AÇO RÁPIDO 10MM.,5,adriana.carmo@cabo.ifpe.edu.br,both,0
262,30.42.789,BROCA DE AÇO RÁPIDO 12 MM,UN,5,42 - FERRAMENTAS,BROCA DE AÇO RÁPIDO 12 MM,5,adriana.carmo@cabo.ifpe.edu.br,both,0
263,30.42.790,BROCA AÇO RÁPIDO DE 8 MM,UN,5,42 - FERRAMENTAS,BROCA AÇO RÁPIDO DE 8 MM,5,adriana.carmo@cabo.ifpe.edu.br,both,0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.07.1462,07 - GENEROS DE ALIMENTACAO,Açucar cristal 1KG,93,93,0,adequado
1,30.07.1464,07 - GENEROS DE ALIMENTACAO,Café em pó 250g,208,208,0,adequado
2,30.07.451,07 - GENEROS DE ALIMENTACAO,"ADOÇANTE, ASPECTO FÍSICO LÍQUIDO SUCRALOSE.",10,10,0,adequado
3,30.16.1065,16 - MATERIAL DE EXPEDIENTE,"ESTOJO PARA DVD, EM PVC 19 X 13,5 X 1,3CM - TR...",195,195,0,adequado
4,30.16.1069,16 - MATERIAL DE EXPEDIENTE,PRANCHETA 1/2 OFÍCIO CRISTAL C/ PRENDEDOR PLÁS...,22,22,0,adequado
...,...,...,...,...,...,...,...
260,30.42.12,42 - FERRAMENTAS,BROCA DE VIDIA 12MM,5,5,0,adequado
261,30.42.213,42 - FERRAMENTAS,BROCA DE AÇO RÁPIDO 10MM.,5,5,0,adequado
262,30.42.789,42 - FERRAMENTAS,BROCA DE AÇO RÁPIDO 12 MM,5,5,0,adequado
263,30.42.790,42 - FERRAMENTAS,BROCA AÇO RÁPIDO DE 8 MM,5,5,0,adequado
